# **CCMS Analysis : Challlenge 1**
### How to detect/  are not identify the defective/non-functional lights, numver of lights connected to a particular ccms that are not functioning and the wattage of these faulty lights.

## 1. Importing the required libraries

In [0]:
#imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import dateutil
%matplotlib inline

In [85]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. Loading the dataset

In [0]:
#reading file
df = pd.read_excel("/content/drive/My Drive/Dummy data/Load Survey Data (6).xlsx")

## 3. Data PreProcessing and cleaning

In [0]:
#converting timestamp oject to datetime object
df['Timestamp'] = pd.to_datetime(df.Timestamp)

#list of columns to drop
drop_columns = ['Vr (Volt)','Vy (Volt)','Vb (Volt)','Ir (Amp)','Iy (Amp)','Ib (Amp)','Freq (Hz)']

#dropped columns dataframe
df = df.drop(drop_columns,axis=1)

#changing column names
df.columns = ['CCMS','timestamp','kwh','kvah','pf_total','total_load']
df['time_hour'] = df.timestamp.apply(lambda x: x.hour)

In [88]:
df.columns

Index(['CCMS', 'timestamp', 'kwh', 'kvah', 'pf_total', 'total_load',
       'time_hour'],
      dtype='object')

In [89]:
df.dtypes

CCMS                  object
timestamp     datetime64[ns]
kwh                  float64
kvah                 float64
pf_total              object
total_load           float64
time_hour              int64
dtype: object

In [90]:
df.head()

,CCMS,timestamp,kwh,kvah,pf_total,total_load,time_hour
0,SMR03-0219-0252,2019-05-01 23:00:00,2.732,2.752,0.955 Ld,5.17,23
1,SMR03-0219-0252,2019-05-01 22:00:00,5.108,5.143,0.987 Ld,5.17,22
2,SMR03-0219-0252,2019-05-01 21:00:00,5.120,5.156,0.987 Ld,5.17,21
3,SMR03-0219-0252,2019-05-01 20:00:00,4.985,5.025,0.985 Ld,5.17,20
4,SMR03-0219-0252,2019-05-01 19:00:00,0.372,0.375,0.87 Lag,5.17,19


## Basic data description

In [100]:
df.describe()

,kwh,kvah,total_load,time_hour
count,742.000000,742.000000,7.420000e+02,742.000000
mean,2.139387,2.153534,5.170000e+00,11.505391
std,2.296270,2.311301,4.799399e-14,6.935356
min,0.000000,0.000000,5.170000e+00,0.000000
25%,0.000000,0.000000,5.170000e+00,5.250000
50%,0.480000,0.484000,5.170000e+00,12.000000
75%,4.493000,4.520000,5.170000e+00,17.750000
max,5.453000,5.489000,5.170000e+00,23.000000


In [92]:
df.head()

,CCMS,timestamp,kwh,kvah,pf_total,total_load,time_hour
0,SMR03-0219-0252,2019-05-01 23:00:00,2.732,2.752,0.955 Ld,5.17,23
1,SMR03-0219-0252,2019-05-01 22:00:00,5.108,5.143,0.987 Ld,5.17,22
2,SMR03-0219-0252,2019-05-01 21:00:00,5.120,5.156,0.987 Ld,5.17,21
3,SMR03-0219-0252,2019-05-01 20:00:00,4.985,5.025,0.985 Ld,5.17,20
4,SMR03-0219-0252,2019-05-01 19:00:00,0.372,0.375,0.87 Lag,5.17,19


## Multiple analysis (sub analysis)
-  Time period when the street lights are off (plot using bar chart)
-  Random fluctuations of streetlights due to various reason like powercut etc (plot using scatter plot)
-  streetlights pulling load lesser than the mean threshhold value (average KW/hr consumption +- std deviation)

## CCMS off_time and uptime dataframes

- In this section we will explore the dataset to find the duration of uptime and downtimes of the ccms 

### CCMS off-time_values (excluding faulty values)

In [0]:
off_time_df = df[df.time_hour >= 8]
off_time_df = off_time_df[off_time_df.time_hour <= 17]
#off_time_df.describe()

In [94]:
off_time_df.head()

,CCMS,timestamp,kwh,kvah,pf_total,total_load,time_hour
6,SMR03-0219-0252,2019-05-01 17:00:00,0.0,0.0,--,5.17,17
7,SMR03-0219-0252,2019-05-01 16:00:00,0.0,0.0,--,5.17,16
8,SMR03-0219-0252,2019-05-01 15:00:00,0.0,0.0,--,5.17,15
9,SMR03-0219-0252,2019-05-01 14:00:00,0.0,0.0,--,5.17,14
10,SMR03-0219-0252,2019-05-01 13:00:00,0.0,0.0,--,5.17,13


### CCMS uptime_values (including faulty values)

In [95]:
on_time_df = df[(df.time_hour >= 18) | (df.time_hour <= 7)]
#on_time_df.describe()
on_time_df.head()

,CCMS,timestamp,kwh,kvah,pf_total,total_load,time_hour
0,SMR03-0219-0252,2019-05-01 23:00:00,2.732,2.752,0.955 Ld,5.17,23
1,SMR03-0219-0252,2019-05-01 22:00:00,5.108,5.143,0.987 Ld,5.17,22
2,SMR03-0219-0252,2019-05-01 21:00:00,5.120,5.156,0.987 Ld,5.17,21
3,SMR03-0219-0252,2019-05-01 20:00:00,4.985,5.025,0.985 Ld,5.17,20
4,SMR03-0219-0252,2019-05-01 19:00:00,0.372,0.375,0.87 Lag,5.17,19


### Time period where lights were off when it should have been on during uptime duration!
* Can be down due to external factors eg. light outage, etc.
* Something is faulty/ not working as expected

In [96]:
faulty_values = on_time_df[(on_time_df.kwh == 0) & (on_time_df.time_hour != 18) & (on_time_df.time_hour != 7)]
faulty_values.head()

,CCMS,timestamp,kwh,kvah,pf_total,total_load,time_hour
355,SMR03-0219-0252,2019-04-17 02:00:00,0.0,0.0,--,5.17,2
356,SMR03-0219-0252,2019-04-17 01:00:00,0.0,0.0,--,5.17,1
379,SMR03-0219-0252,2019-04-16 02:00:00,0.0,0.0,--,5.17,2


* here we can conclude that the ccms has not logged the data due to power outage/grid outage

## After adding sensors to streetlights and connecting them to CCMS to send data to the server

In [97]:
df_new = pd.read_excel("/content/drive/My Drive/Dummy data/Untitled spreadsheet.xlsx")
df_new.columns 

Index(['CCMS ID', 'SL ID', 'Current', 'Voltage', 'Resistance', 'Power'], dtype='object')

In [107]:
df_new.columns = ['ccms','slid','current','voltage','resistance','power']
df_new

,ccms,slid,current,voltage,resistance,power
0,a,1,2,2,4.00,1
1,a,2,4,25,6.25,100
2,a,3,2,2,4.00,1
3,a,4,2,2,4.00,1
4,a,5,2,2,4.00,1
5,a,6,2,2,4.00,1
6,a,7,2,2,4.00,1
7,a,8,2,2,4.00,1
8,a,9,2,2,4.00,1
9,a,10,2,2,4.00,1


In [0]:
ccms_fault =[]
slid_fault = []
for index,row in df_new.iterrows():
    if row.power >= (df_new.power.mean()+df_new.power.std()) or row.power <= (df_new.power.mean()-df_new.power.std()):
        ccms_fault.append(row.ccms)
        slid_fault.append(row.slid)

In [106]:
ccms_fault

['a']

In [105]:
slid_fault

[2]